In [ ]:
import os
import subprocess
from glob import glob
from pathlib import Path
import csv

blast_dir = "MetaGeneMark/filtered_blast_results"
fasta_dir = "MetaGeneMark/nt_fasta_results"
reads_dir = "filtered_samples_bowtie2"
output_dir = "ORF_processing2"
Path(output_dir).mkdir(exist_ok=True)

def parse_best_annotations(blast_file):
    """
    Возвращает словарь {ORF_id: best_hit_name_by_pident}
    """
    best_hits = {}
    with open(blast_file) as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) < 3:
                continue
            orf_id, hit_name, pident = parts[0], parts[1], float(parts[2])
            if orf_id not in best_hits or best_hits[orf_id][1] < pident:
                best_hits[orf_id] = (hit_name, pident)
    return {orf: hit[0] for orf, hit in best_hits.items()}

def extract_fasta(input_fasta, ids, output_fasta):
    with open(input_fasta) as infile, open(output_fasta, "w") as outfile:
        write = False
        for line in infile:
            if line.startswith(">"):
                orf_id = line[1:].split()[0]
                write = orf_id in ids
                if write:
                    outfile.write(line)
            elif write:
                outfile.write(line)

def build_bowtie2_index(fasta_file, index_prefix):
    subprocess.run(["bowtie2-build", fasta_file, index_prefix], check=True)

def align_and_count(index_prefix, r1_file, r2_file, output_base):
    sam_file = f"{output_base}.sam"
    bam_file = f"{output_base}.sorted.bam"
    idxstats_file = f"{output_base}_idxstats.tsv"

    # bowtie2
    subprocess.run([
        "bowtie2", "-p", "32", "-x", index_prefix,
        "-1", r1_file, "-2", r2_file,
        "-S", sam_file
    ], check=True)

    # SAM to BAM
    subprocess.run(["samtools", "view", "-bS", sam_file], stdout=open(f"{output_base}.bam", "wb"), check=True)

    # Sort BAM 
    subprocess.run(["samtools", "sort", "-@", "32", f"{output_base}.bam", "-o", bam_file], check=True)
    subprocess.run(["samtools", "index", bam_file], check=True)

    # Count
    with open(idxstats_file, "w") as out:
        subprocess.run(["samtools", "idxstats", bam_file], stdout=out, check=True)

    # Cleanup
    os.remove(sam_file)
    os.remove(f"{output_base}.bam")

    return idxstats_file


blast_files = glob(os.path.join(blast_dir, "*_blast_filtered.txt"))
for blast_file in blast_files:
    sample_name = Path(blast_file).stem.replace("_orf_aa_blast_filtered", "")
    nt_fasta = os.path.join(fasta_dir, f"{sample_name}_orf_nt.fasta")
    annotations = parse_best_annotations(blast_file)
    orf_ids = list(annotations.keys())

    filtered_fasta = os.path.join(output_dir, f"{sample_name}_filtered_orfs.fasta")
    extract_fasta(nt_fasta, set(orf_ids), filtered_fasta)

    index_prefix = os.path.join(output_dir, f"{sample_name}_index")
    build_bowtie2_index(filtered_fasta, index_prefix)

    r1 = glob(os.path.join(reads_dir, f"{sample_name}*_R1_*.fq.gz"))[0]
    r2 = glob(os.path.join(reads_dir, f"{sample_name}*_R2_*.fq.gz"))[0]
    output_base = os.path.join(output_dir, sample_name)

    idxstats_file = align_and_count(index_prefix, r1, r2, output_base)

    # Переводим idxstats в аннотированный tsv
    final_output = f"{output_base}_annotated_counts.tsv"
    with open(idxstats_file) as fin, open(final_output, "w", newline="") as fout:
        reader = csv.reader(fin, delimiter="\t")
        writer = csv.writer(fout, delimiter="\t")
        writer.writerow(["orf_id", "gene_name", "length", "mapped_reads"])
        for row in reader:
            orf_id, length, mapped, _ = row
            if orf_id == "*" or orf_id not in annotations:
                continue
            gene_name = annotations[orf_id]
            writer.writerow([orf_id, gene_name, length, mapped])

print("Выравнивание, аннотация и подсчёты завершены.")


Settings:
  Output files: "ORF_processing2/NTZ803_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ORF_processing2/NTZ803_filtered_orfs.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 129621
Using parameters --bmax 97216 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 97216 --dcv 1024
Construc

Building a SMALL index


Exiting Ebwt::buildToDisk()
Returning from initFromVector
Wrote 4380950 bytes to primary EBWT file: ORF_processing2/NTZ803_index.1.bt2
Wrote 129628 bytes to secondary EBWT file: ORF_processing2/NTZ803_index.2.bt2
Re-opening _in1 and _in2 as input streams
Returning from Ebwt constructor
Headers:
    len: 518487
    bwtLen: 518488
    sz: 129622
    bwtSz: 129622
    lineRate: 6
    offRate: 4
    offMask: 0xfffffff0
    ftabChars: 10
    eftabLen: 20
    eftabSz: 80
    ftabLen: 1048577
    ftabSz: 4194308
    offsLen: 32406
    offsSz: 129624
    lineSz: 64
    sideSz: 64
    sideBwtSz: 48
    sideBwtLen: 192
    numSides: 2701
    numLines: 2701
    ebwtTotLen: 172864
    ebwtTotSz: 172864
    color: 0
    reverse: 0
Total time for call to driver() for forward index: 00:00:00
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00

4814556 reads; of these:
  4814556 (100.00%) were paired; of these:
    4748294 (98.62%) aligned concordantly 0 times
    65262 (1.36%) aligned concordantly exactly 1 time
    1000 (0.02%) aligned concordantly >1 times
    ----
    4748294 pairs aligned concordantly 0 times; of these:
      7287 (0.15%) aligned discordantly 1 time
    ----
    4741007 pairs aligned 0 times concordantly or discordantly; of these:
      9482014 mates make up the pairs; of these:
        9469331 (99.87%) aligned 0 times
        12261 (0.13%) aligned exactly 1 time
        422 (0.00%) aligned >1 times
1.66% overall alignment rate
[bam_sort_core] merging from 0 files and 32 in-memory blocks...


Settings:
  Output files: "ORF_processing2/UUS772_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ORF_processing2/UUS772_filtered_orfs.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 68140
Using parameters --bmax 51105 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 51105 --dcv 1024
Construct

...

3484637 reads; of these:
  3484637 (100.00%) were paired; of these:
    3462352 (99.36%) aligned concordantly 0 times
    20839 (0.60%) aligned concordantly exactly 1 time
    1446 (0.04%) aligned concordantly >1 times
    ----
    3462352 pairs aligned concordantly 0 times; of these:
      2832 (0.08%) aligned discordantly 1 time
    ----
    3459520 pairs aligned 0 times concordantly or discordantly; of these:
      6919040 mates make up the pairs; of these:
        6914380 (99.93%) aligned 0 times
        4102 (0.06%) aligned exactly 1 time
        558 (0.01%) aligned >1 times
0.79% overall alignment rate
[bam_sort_core] merging from 0 files and 32 in-memory blocks...


Settings:
  Output files: "ORF_processing2/CRQ549_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ORF_processing2/CRQ549_filtered_orfs.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 140979
Using parameters --bmax 105735 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 105735 --dcv 1024
Constr

Building a SMALL index


Exiting Ebwt::buildToDisk()
Returning from initFromVector
Wrote 4403165 bytes to primary EBWT file: ORF_processing2/CRQ549_index.1.bt2
Wrote 140984 bytes to secondary EBWT file: ORF_processing2/CRQ549_index.2.bt2
Re-opening _in1 and _in2 as input streams
Returning from Ebwt constructor
Headers:
    len: 563919
    bwtLen: 563920
    sz: 140980
    bwtSz: 140980
    lineRate: 6
    offRate: 4
    offMask: 0xfffffff0
    ftabChars: 10
    eftabLen: 20
    eftabSz: 80
    ftabLen: 1048577
    ftabSz: 4194308
    offsLen: 35245
    offsSz: 140980
    lineSz: 64
    sideSz: 64
    sideBwtSz: 48
    sideBwtLen: 192
    numSides: 2938
    numLines: 2938
    ebwtTotLen: 188032
    ebwtTotSz: 188032
    color: 0
    reverse: 0
Total time for call to driver() for forward index: 00:00:00
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00

4608623 reads; of these:
  4608623 (100.00%) were paired; of these:
    4593700 (99.68%) aligned concordantly 0 times
    14448 (0.31%) aligned concordantly exactly 1 time
    475 (0.01%) aligned concordantly >1 times
    ----
    4593700 pairs aligned concordantly 0 times; of these:
      2359 (0.05%) aligned discordantly 1 time
    ----
    4591341 pairs aligned 0 times concordantly or discordantly; of these:
      9182682 mates make up the pairs; of these:
        9179435 (99.96%) aligned 0 times
        2993 (0.03%) aligned exactly 1 time
        254 (0.00%) aligned >1 times
0.41% overall alignment rate
[bam_sort_core] merging from 0 files and 32 in-memory blocks...


In [11]:
import os
import subprocess
import glob
from pathlib import Path
import csv

blast_dir = "MetaGeneMark/filtered_blast_results"
fasta_dir = "MetaGeneMark/nt_fasta_results"
reads_dir = "filtered_samples_bowtie2"
output_dir = "ORF_processing2"
def load_counts_by_gene(input_dir):
    files = glob.glob(os.path.join(input_dir, "*_annotated_counts.tsv"))
    all_counts = []

    for file in files:
        sample_name = os.path.basename(file).split("_annotated_counts.tsv")[0]
        df = pd.read_csv(file, sep="\t")
        df = df[["gene_name", "mapped_reads"]]
        df = df.groupby("gene_name").sum()
        df.columns = [sample_name]
        all_counts.append(df)

    merged = pd.concat(all_counts, axis=1).fillna(0).astype(int)
    return merged

# Папка с *_annotated_counts.tsv
input_folder = "ORF_processing2"

# Загружаем объединённую таблицу по gene_name
counts = load_counts_by_gene(input_folder)
counts.to_csv("combined_raw_counts.tsv", sep="\t")

